# Interneto sistemų, paslaugų kūrimas

Flask - "mažas" karkasas (angl. microframework) skirtas web sistemų kūrimui. Flask yra pagrįstas Werkzeug (WSGI biblioteka (angl. Web Server Gateway Interface)) ir Jinja 2 (šablonų variklis (angl. template engine)) bibliotekomis. Mažu šis karkasas vadinamas, todėl, jog šis karkasas yra ganėtinai "plikas", neturi kitiems karkasas būdingų funcionalumų, pvz. validatoriai, duomenų bazės palaikymas, migracijos ir t.t. Jūs gaunat pagrindą, o reikiamus funkcionalumus, struktūrą ir pan. kuriatės patys (diegiat reikiamus Flask papildymus).

In [ ]:
# aplikacijos pavyzdys
from flask import Flask # importuojame Flask modulį
app = Flask(__name__) # sukuriamas Flask objektas

@app.route("/") # sukuriama URL taisyklė, kurią pasiekus iškviečiama dekoruojama funkcija
def index(): # funkcija, kuri bus vykdoma pasiekus URL, dažnai vadinama view'u
    return("Hello World") # atsakas (angl. response) duomenys

if __name__ == "__main__":
    app.run() # app.run(host, port, debug, options)

Norint užregistruoti funkciją, kuri bus iškviečiama pasiekus tam tikrą URL, funkcija dekoruojama dekoratoriumi @app.route("/"). Be šio dekoratoriaus tą patį rezultatą galima pasiekti ir panaudojus funkciją app.add_url_rule("/", "index", index)

In [ ]:
# aplikacijos pavyzdys
from flask import Flask # importuojame Flask modulį
app = Flask(__name__) # sukuriamas Flask objektas

def index():
    return("Hello World") # atsakas (angl. response) duomenys

app.add_url_rule("/", "index", index) # sukuriama URL taisyklė, kurią pasiekus iškviečiama nurodyta funkcija

if __name__ == "__main__":
    app.run() # app.run(host, port, debug, options)

In [ ]:
# galima sukurti kiek tik reikia URL taisyklių, taip pat vienai funkcijai nurodyti kelias URL taisykles
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return("""
    <!DOCTYPE html>
      <html>
      <head><title>Hello</title></head>
      <body><h1>Hello, from HTML</h1></body>
      </html>
    """)

@app.route("/career/")
def career():
    return("Career Page")

@app.route("/feedback/")
def feedback():
    return("Feedback Page")

@app.route('/first/first/')
@app.route('/second/first/')
def numbers():
    return("Multiple URLS")

if __name__ == "__main__":
    app.run()

### Dinaminiai URL, parametrai

Norint sukurti dinaminius URL (funkcijai perduoti parametrus) aprašant URL naudojamas sekantis formatas: "/user/<id>/". Toks užrašymas nurodo kad funkcijai bus perduodamas parametras id, kurio tipas yra bet koks. Toks URL užrašymas įvykdys funkciją kai bus pasiekiamas adresas /users/1/, /users/100/, /users/pirmas/, /users/antras/. Kaip matom id laukas galės įgyti tiek int tiek string tipo reikšmes. Dėl to galima naudoti tokį URL formatą <konvertatorius:kintamojo pavadinimas>, kuris nurodo kokio tipo kintamojo tikimės, galimos reikšmės: string, int, float, path, uuid.

In [ ]:
# dinaminis URL
from flask import Flask

app = Flask(__name__)

@app.route("/user/<id>/")
def user_profile(id):
    return("Profile page of user #{}".format(id))

if __name__ == "__main__":
    app.run()

In [ ]:
# dinaminis URL su parametro tipo tikrinimu
from flask import Flask

app = Flask(__name__)

@app.route("/user/<int:id>/")
def user_profile(id):
    return("Profile page of user #{}".format(id))

if __name__ == "__main__":
    app.run()

### Debug būsena

Pagal nutylėjimą debug būsena yra išjungta. Jei įvyksta klaida programiniame kode, vartotojui grąžinamas 500 atsakas, be papildomų paaiškinimų. Įsijungus debug būseną į ekraną grąžinama pilna informacija apie klaidą. Taip pat jei įjungta ši būsena, po kiekvieno failo pakoregavimo ir išsaugojimo serveris yra perleidžiamas iš naujo.

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def index():
    print(i)
    return("Index")

if __name__ == "__main__":
    app.run(debug=True) # True/False

### URL ir funkcijų sąrašas

app objekta turi metodą app.url_map, kuris grąžina sąrašą URL->funkcija susiejimų.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return("Home Page")

@app.route("/career/")
def career():
    return("Career Page")

@app.route("/feedback/")
def feedback():
    return("Feedback Page")

@app.route('/first/first/')
@app.route('/second/first/')
def numbers():
    return("Multiple URLS")

print(app.url_map)

### Kontekstas (angl. contexts) Flask karkase

Kaip pastebėjote, visose view funkcijose nėra paduodamas pačios užklausos (angl. request) parametras. Šis parametras yra pasiekiamas visose funkcijose. Gali susidaryti įspūdis, jog jis yra globalus, tačiau taip nėra, nes daugiagijėse (angl. multithread) programose kiltų problemų (vienu metu paleistos kelios applikacijos, pvz. naudojant DispatcherMiddleware), kokia užklausa priklauso kokiai gijai (angl. thread), todėl Flask naudoja kintamuosius, kurie yra lokalūs kiekvienai gijai (angl. thread locals), turi savo kontekstą.

Flask turi kelis skirtingus kontekstus: aplikacijos ir užklausos. Aplikacijos kontekstas skirtas saugoti bendrus duomenis, tokius kaip duomenų bazės prisijungimus, kitus nustatymus, o užklausos kontekste saugomi duomenys skirti kiekvienai užklausai.

http://flask.pocoo.org/docs/0.12/appcontext/
http://flask.pocoo.org/docs/0.12/reqcontext/

Flask karkase aplikacijos ir užklausos kontekstai yra aktyvuojami/sukuriami kai gaunama nauja užklausa, ir deaktyvuojami, kai užklausa yra apdorota.

In [ ]:
# užklausos konteksto pavyzdys
from flask import Flask, request

app = Flask(__name__)

@app.route("/")
def requestdata():
    return("Hello! Your IP is {} and you are using {}: ".format(request.remote_addr, request.user_agent))

if __name__ == "__main__":
    app.run()

### Užklausos atsakymas (angl. response)

Flask karkase yra trys atsakymo tipai:
1. Paprastas tekstas arba naudojant šablonų variklį.
2. Response objektas.
3. Sąrašas iš (atsakyams, būsena, antraštės) arba (atsakymas, antraštės/būsena) (angl. (response, status, headers) or (response, headers/status)).

Kai grąžinamas tekstas Flask automatiškai tą tekstą paverčia į Response objektą (naudojant funkciją make_response()) su HTTP būsena 200 ir antrašte content-type = text/html. Norint pakeisti grąžinamus parametrus, galima išsikviesti make_response() funkciją ir atlikti jo grąžinto objekto korekcijas.

In [ ]:
# make_response funkcijos pavyzdys
from flask import Flask, make_response

@app.route("/books/<genre>")
def books(genre):
    res = make_response("All Books in {} category".format(genre))
    res.headers["Content-Type"] = "text/plain"
    res.headers["Server"] = "VU"
    return(res)

In [ ]:
# grąžinamas atsakymas su būsena 400
@app.route("/")
def http_404_handler():
    return make_response("<h2>404 Error</h2>", 400)

In [ ]:
# sausainėlių (angl. cookies) nustatymas
@app.route("/set-cookie")
def set_cookie():
    res = make_response("Cookie setter")    
    res.set_cookie("favorite-color", "skyblue")
    res.set_cookie("favorite-font", "sans-serif")
    # res.set_cookie("favorite-color", "skyblue", 60*60*24*15) # cookies su galiojimo laiku
    # res.set_cookie("favorite-font", "sans-serif", 60*60*24*15) # cookies su galiojimo laiku
    return res

In [ ]:
# atsakymas kaip sąrašas
@app.route("/")
def http_500_handler():
    return ("<h2>500 Error</h2>", 500)
    # return("## Heading", 200, {'Content-Type': 'text/markdown'}) # grąžinamas atsakymas su antraštėmis

### Užklausos peradresavimas

In [ ]:
@app.route("/transfer")
def transfer():
    return "", 302, {"location": "http://localhost:5000/login"}

In [ ]:
from flask import Flask, redirect

@app.route("/transfer")
def transfer():
    return redirect("http://localhost:5000/login")
    # return redirect("http://localhost:5000/login", code=301) # peradresavimas su būsena

### Spąstų vietos (angl. hook points)

Dažnai web aplikacijose reikia atlikti tam tikrus veiksmus prieš kiekvieną užklausą, pvz. kliento IP adreso užloginimas, arba vartotojo autentikavimas prieš parodant jiems turinį ir pan. Flask turi kelis dekoratorius, kurie gali pagelbėti atlikti šiuos veiksmus:

* before_first_request - vykdys funkciją prieš pirmą užklausą.
* before_request - vykdys funkciją prieš kiekvieną užklausą.
* after_request - vykdys funkciją po užklausos vykdymo. Nėra vykdoma įvykus nesuvaldytai klaidai (angl. unhandled exception). Funkcija privalo priimti atsakymo objektą (angl. response objext), o grąžinti tą patį arba naują atsakymo objektą.
* teardown_request - panašu į after_request dekoratorių, tik šį funkciją bus vykdoma visada nepaisant ar buvo klaida ar ne.

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.before_first_request
def before_first_request():
    print("before_first_request() called")

@app.before_request
def before_request():
    print("before_request() called")

@app.after_request
def after_request(response):
    print("after_request() called")
    return(response)

@app.route("/")
def index():
    print("index() called")
    return("<p>Testings Request Hooks</p>")

@app.teardown_request
def teardown_request(response):
    print("teardown_request() called")
    return(response)

if __name__ == "__main__":
    app.run(debug=True)

### Užklausų atšaukimas

Užklausų atšaukimui naudojama funkcija abort() su grąžinamu atsakymo kodu. Tokiu atveju grąžinamas tas pats atsakymas skirtingų klaidų atveju. Norint sukurti atskirus klaidų atsakymus naudojamas errorhandler dekoratorius.

In [ ]:
from flask import Flask, abort

@app.route("/")
def index():
    abort(404)
    # tolimesnis kodas nebus vykdomas

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
@app.errorhandler(404)
def http_404_handler(error):
    return("<p>HTTP 404 Error Encountered</p>", 404)

@app.errorhandler(500)
def http_500_handler(error):
    return("<p>HTTP 500 Error Encountered</p>", 500)

### Šablonai (angl. templates)

Šablonas - HTML failas, kuris sudarytas iš statinio HTML kodo, bei dinaminių žymų, kurios bus žinomas užklausos apdorojimo metu. Procesas, kurio metu dinaminė žymė, pakeičiama į HTML kodą vadinamas šablono generavimu (angl. template rendering). 

Jinja šablonų variklis (angl. template engine) - vienas populiariausių šablonų generavimo variklių, tad jis ir bus naudojamas.

Pagal nutylėjimą Flask ieško šablonų aplikacijos pakatalogyje "templates". Pakeisti šablonų katalogą galima nurodant "template_folder" parametro reikšmę

```python
app = Flask(__name__, template_folder="jinja_templates")
```

Sukurkime failą index.html, su kodu:
```html
<html>
    <head>
        <title>Title</title>
    </head>
    <body>
        <p>Name: {{ name }}</p>
    </body>
</html>
```

Kaip matome šalia standarinių HTML žymių naudojamas ir dinaminis komponentas {{name}}. Tarp {{ ir }} esantis pavadinimas nurodo kintamojo pavadinimą, kurio reikšmę reikia patalpinti nurodytoje vietoje.

Flask turi funkciją "render_template()", kuri atlieka šablono generavimą. Kviečiant šią funkciją jei nurodoma kokį šabloną krauti, bei su kokiais duomenimis.

In [ ]:
from flask import Flask, request, render_template
import os
app = Flask(__name__, template_folder=os.path.join("files", "templates"))

@app.route('/')
def index():
    return render_template("index.html", name="Jerry")

if __name__ == "__main__":
    app.run()

Funkcijai "render_template" galima paduodi daugiau duomenų.
```python
name, age, profession = "Jerry", 24, 'Programmer'
template_context = dict(name=name, age=age , profession=profession)
return render_template('index.html', **template_context)
```

Jei funkcijai nenurodomi duomenys, kuriuos naudoti generuojant šabloną, tai šablonas bus sugeneruotas su tuščiais duomenimis.

Daugiau apie Jinja http://jinja.pocoo.org/docs/2.10/

Pagrindinės Jinja komandos.

IF
```html
{% if user %}
    <p>Vartotojas prisijungęs</p>
{% else %}
    <p>Vartotojas neprisijungęs</p>
{% endif %}
```

```html
{% if user == 'Jonas' %}
    <p>Vartotojas prisijungęs</p>
{% else %}
    <p>Vartotojas neprisijungęs</p>
{% endif %}
```

FOR
```html
<ul>
{% for user in user_list %}
    <li>{{ user }}</li>
{% endfor %}
</ul>
```

Objektų atributų pasiekiamumas
```html
{{ obj.i }}
```

Šablonų įterpimas - include komanda.
```html
{% include 'path/to/template' %}
```

Šablonų paveldimumas - extends komanda (naudojama su block komanda).
```html
{% extends 'base.html' %}

{% block title %}
{% endblock %}
```

In [ ]:
from flask import Flask, request, render_template
import os
app = Flask(__name__, template_folder=os.path.join("files", "templates"))

@app.route('/base')
def base():
    return render_template("base.html")

@app.route('/index')
def index():
    return render_template("index.html", title="Python programavimas", user=None, students_list=[("Marius", "Mariauskas"), ("Petras", "Petrauskas"), ("Ignas", "Ignauskas")])

if __name__ == "__main__":
    app.run()

index.html
```html
{% extends 'base.html' %}

{% block title %}
{{title}}
{% endblock %}

{% block content %}
	{% if user %}
		<p>Sveikas {{ user }}</p>
	{% else %}
		<p>Vartotojas nežinomas</p>
	{% endif %}

   	<br>

	{% include 'include.html' %}

	<table>
		{% for student in students_list %}
    		<tr>
    			<td>{{ student[0] }}</td>
    			<td>{{ student[1] }}</td> 
  			</tr>
		{% endfor %}
	</table>
{% endblock %}
```

base.html
```html
<html>
    <head>
        <title>{% block title %}Default Title{% endblock %}</title>
    </head>
    <body>
        <nav>
            <a href="/home">Home</a>
            <a href="/blog">Blog</a>
            <a href="/contact">Contact</a>  
        </nav>
    	{% block content %}
		{% endblock %}
    </body>
</html>
```

include.html
```html
<p>Included from another file.</p>
```

### URL nuorodų generavimas

Kietai įrašyti (angl. harcode) URL nuorodas programiniame kode yra bloga praktika. Todėl visada reikia naudoti funkciją "url_for()" tiek šablonuose, tiek programiniam kode. Jei adresas yra su parametru, t.y. dinaminis, parametro reikšmę galima nurodyti šioje funkcijoje, pvz.: adresas /post/<int:id>, tai URL generavimo funkcijoje nurodysime url_for('post', id=100).

In [ ]:
from flask import Flask, request, render_template, url_for
import os
app = Flask(__name__, template_folder=os.path.join("files", "templates"))

@app.route('/login')
def login():
    return "<p>Please login</p>"

@app.route('/urls')
def urls():
    return render_template("urls.html", login_url=url_for("login"))

if __name__ == "__main__":
    app.run()

### Statiniai failai

Statiniai failai (angl. static files) tai tokie failai kurie dažnai nesikeičia, pvz.: css, js, font failai ir pan. Pagal nutylėjimą Flask ieško statinių failų aplikacijos pakatalogyje "static", tačiau jį galima perrašyti naudojant parametrą "static_folder".

```python
app = Flask(__name__, static_folder="static_dir")
```

Stainius failus Flask automatiškai leidžia pasiekti per adresą /static/<failo_pavadinimas>, pvz.: http://127.0.0.1:5000/static/style.css.

In [ ]:
from flask import Flask, request, render_template, url_for
import os
app = Flask(__name__, template_folder=os.path.join("files", "templates"), static_folder=os.path.join("files", "static"))

@app.route('/')
def index():
    return render_template("static_file_index.html")

if __name__ == "__main__":
    app.run()

### Formų valdymas

Formos yra labai svarbi web aplikacijų dalis. Jos leidžia vykdyti duomenų įvedimą. Dirbti su formomis yra sudėtinga, nes reikia atkreipti dėmesį į daug detalių: CSRF, XSS, SQL įdėjimas (angl. injection), duomenų validavimas tiek klientinėje dalyje, tiek serveryje ir pan. Biblioteka WTForms gali palenginti visą šį darbą (https://flask-wtf.readthedocs.io/en/stable/).

In [ ]:
from flask import Flask, render_template, request
import os

app = Flask(__name__, template_folder=os.path.join("files", "templates"))

@app.route("/login/", methods=["post", "get"])
def login():
    message = ""
    if request.method == "POST":
        username = request.form.get("username")
        password = request.form.get("password")

        if username == "root" and password == "pass":
            message = "Correct username and password"
        else:
            message = "Wrong username or password"

    return render_template("login.html", message=message)

if __name__ == "__main__":
    app.run()

### Slapukai (angl. cookies)

Kol kas naršyklė siunčia užklausą, gavę ją sugeneruojama atsakymo tekstą ir grąžiname jį naršyklei. Viskas yra gan paprasta. Svarbu paminėti, jog HTTP protokolas yra be būsenos (angl. stateless), t.y. gavus kelias užklausas iš kliento yra neįmanoma pasakyti, kad jos yra iš to paties kliento, taip pat neįmanoma pasakyti ar užklausiate puslapio pirmą ar trečią kartą. Šiaip problemai spręsti yra naudojami slapukai (angl. cookies) ir sesijos (angl. sessions).

Slapukai - duomenys, kuriuos serveris išsaugo naršyklėje. Pagal wikipedia - mažas tekstinis dokumentas, turintis unikalų identifikacijos numerį, kuris yra perduodamas iš interneto tinklalapio į lankytojo kompiuterio kietąjį diską.

Kaip tai veikia:
1. Naršyklė siunčia web puslapio užklausą į serverį.
2. Serveris siunčia naršyklės užklaustą web puslapį kartu su slapukais.
3. Gavus atsakymą naršyklė atvaizduoja web puslapį klientui ir išsaugo slapuko duomenis kietam diske.
4. Sekančios naršyklės užklausos į serverį bus siunčiamos su visais slapukais (slapukų antraštėje (angl. cookie header)), tai tęsis tol, kol slapukas galios.

Slapukui nustatyti Flask yra naudojama funkcija set_cookie().

```python
set_cookie(key, value="", max_age=None)
```

key - raktas, value - reikšmė, max_age - laikas sekundėmis, kiek laiko galios slapukas.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, make_response

app = Flask(__name__)

@app.route("/cookie/")
def cookie():
    res = make_response("Setting a cookie")
    res.set_cookie("foo", "bar", max_age=60*60*24*365*2)
    return res

if __name__ == "__main__":
    app.run()

Naršyklėje:

![alt text](files/images/cookies.png "Logo Title Text 1")

Slapukų nuskaitymui naudojamas request kintamojo atributas cookie. Tai yra žodyno tipo kintamasis, kuriame saugomi visi slapukai, kuriuos siunčia naršyklė. Svarbu paminėti jog kintamasis request yra pasiekiamas ir iš šablonų, tad galima slapukų informaciją pasiekti tiesiai šablone.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, make_response

app = Flask(__name__)

@app.route("/cookie/")
def cookie():
    if not request.cookies.get("foo"):
        res = make_response("Setting a cookie")
        res.set_cookie("foo", "bar", max_age=60*60*24*365*2)
    else:
        res = make_response("Value of cookie foo is {}".format(request.cookies.get("foo")))
    return res

if __name__ == "__main__":
    app.run()

Norint pašalinti slapukus naudojama funkcija set_cookie() tik parametro max_age reikšmė nustatoma 0.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, make_response

app = Flask(__name__)

@app.route("/cookie/")
def cookie():
    if not request.cookies.get("foo"):
        res = make_response("Setting a cookie")
        res.set_cookie("foo", "bar", max_age=60*60*24*365*2)
    else:
        res = make_response("Value of cookie foo is {}".format(request.cookies.get("foo")))
    return res

@app.route("/delete-cookie/")
def delete_cookie():
    res = make_response("Cookie Removed")
    res.set_cookie("foo", "bar", max_age=0)
    return res

if __name__ == "__main__":
    app.run()

Svarbu žinoti, jog:
1. Slapukai yra nesaugus būdas informacijos saugojimui.
2. Slapukai gali būti uždrausti naršyklės.
3. Slapukuose galima saugoti iki 4 KB informacijos.
4. Visi slapukai yra siunčiami su užklausa, tai padidina perduodamos informacijos kiekius.

### Sesijos (angl. sessions)

Sesijos - tai būdas saugoti informaciją tarp užklausų. Jis gan panašus į slapukus. Prieš pradedant naudoti slapukus, būtina Flask nustatyti slaptą raktą, kuriuo bus šifruojami slapukai. Sesijų objektas session naudojamas gauti arba nustatyti sesijos kintamųjų reikšmes, taip pat jis veikia kaip žodynas, tik turi galimybę sekti modifikacijas.

Kai naudojamos sesijos, naršyklėje informacija yra saugoma kaip slapukas ir vadinama sesijos slapuku (angl. session cookie). Kitaip nei paprastas slapukas, sesijos slapukas yra šifruotas. Flask kriptografiškai (angl. Flask Cryptographically) pasirašo kiekvieną sesijos slapuką. Tai reiškia, jog visi gali matyti sesijos slapuko informaciją, tačiau niekas negali jos koreguoti, jei neturi slapto rakto, kuris buvo naudotas pasirašant slapuką. Turint nustatytą sesijos slapuką visos sekančios užklausos į serverį atlieka slapuko patikrinimą ir jį iššifruoja, naudojant nustatytą slaptą raktą. Jei sesijos slapuko iššifruoti nepavyksta, naujas sesijos slapukas yra siunčiamas į kliento naršyklę. 

Flask saugo visą sesijos informaciją kliento kompiuteryje, tai vadinama kliento pusės sesija (angl. client-side sessions), kai tuo tarpu PHP, saugo tik ID, o pati informacija yra saugoma serveryje, tai vadinama serverio pusės sesija (angl. server-side sessions).

Norint Flask naudoti sesijas serverio pusėje, galima naudotis trečių šalių bibliotekomis, tokiomis kaip Flask-Session arba Flask-KVSession.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, make_response, session

app = Flask(__name__)

app.secret_key = "dsfdsafdasf98asd7f89sd7fWX/,?RT"
#arba
app.config['SECRET_KEY'] = "dsfdsafdasf98asd7f89sd7fWX/,?RT"

@app.route("/visits-counter/")
def visits():
    if "visits" in session:
        session["visits"] = session.get("visits") + 1  # sesijos reikšmės nuskaitymas ir koregavimas
    else:
        session["visits"] = 1 # sesijos reikšmės nustatymas
    return "Total visits: {}".format(session.get("visits"))

@app.route("/delete-visits/")
def delete_visits():
    session.pop("visits", None) # reikšmės pašalinimas
    return "Visits deleted"

if __name__ == "__main__":
    app.run()

Pagal nutylėjimą sesijos slapukai galioja tok kol naršyklė neuždaroma. Norint juo išlaikyti ir po naršyklės uždarymo, reikia session objektui nustatyti parametro permanent reikšmę į True. Tada slapukas galio tiek laiko kiek nurodyta aplikacijos nustatymuose permanent_session_lifetime.

```python
app.permanent_session_lifetime = datetime.timedelta(days=365)
# app.config['PERMANENT_SESSION_LIFETIME'] = datetime.timedelta(days=365) # you can also do this
```

Sesijos objektas turi parametrą modified, kuris nurodo, jog sesijos pakeitimus reikia nusiųsti klientui. Tai aktualu, kai sesijoje saugomas koks nors redaguojamas duomenų tipas, kaip sąrašas. Pakeitus sąrašo kažkurio elemento reikšmę reikia nustatyti modified = True, jog Flask pakeitimus nusiųstų klientui.
```python
session["cart"]["pineapples"] = "100"
session.modified = True
```

### SQLAlchemy naudojimas su Flask

Norint naudoti SQLAlchemy su Flask patartina naudotis biblioteka Flask-SQLAlchemy, kuri atlieka SQLAlchemy ir Flask integraciją, bei prideda nemažai papildomo funkcionalumo, apie tai plačiau http://derrickgilland.com/posts/demystifying-flask-sqlalchemy/.

In [ ]:
from flask import Flask, render_template, request
from flask_sqlalchemy import SQLAlchemy
import os
app = Flask(__name__, template_folder=os.path.join("files", "templates"))

# nurodoma kokią DB naudoti SQLAlchemy
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///:memory:" # "sqlite:////tmp/test.db"
app.config['SECRET_KEY'] = "dsfdsafdasf98asd7f89sd7fWX/,?RT"

# sukuriamas duomenų bazės aobjektas
db = SQLAlchemy(app)

# sukuriamas modelis
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

    def __repr__(self):
        return "<User %r>" % self.username

# sukuriame lenteles
db.create_all()

# sukuriami nauji user objektai
admin = User(username="admin", email="admin@example.com")
guest = User(username="guest", email="guest@example.com")

# nauji objektai išsaugomi duomenų bazėje
db.session.add(admin)
db.session.add(guest)
db.session.commit()

@app.route("/users/", methods=["post", "get"])
@app.route("/users/<int:id>", methods=["get"])
def users(id=None):
    if request.method == "POST":
        tmp_user = User(username=request.form.get("username"), email=request.form.get("email"))
        db.session.add(tmp_user)
        db.session.commit()
        
        return render_template("users.html", users=User.query.all())
    elif request.method == "GET":
        users = None
        if id:
            users = [User.query.filter_by(id=id).first()]
        else:
            users = User.query.all()
        
        return render_template("users.html", users=users)

if __name__ == "__main__":
    app.run()

### Interneto paslaugų kūrimas (angl. RESTful API)

